In [1]:
import importlib
from datetime import datetime, timedelta

from poly_api import poly_client


In [4]:
ss_markets = poly_client.get_simplified_markets()
ss_markets["data"][0]

{'condition_id': '0x9deb0baac40648821f96f01339229a422e2f5c877de55dc4dbf981f95a1e709c',
 'rewards': {'rates': None, 'min_size': 0, 'max_spread': 0},
 'tokens': [{'token_id': '14270523446080509320829200481895961480205553513304203367521919818541658424782',
   'outcome': 'Chiefs',
   'price': 1,
   'winner': False},
  {'token_id': '93110170397161149119544349457822484949376809039410140245101963942162463626903',
   'outcome': 'Raiders',
   'price': 0,
   'winner': False}],
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False}

In [5]:
import time


def find_crypto_markets(limit=10, max_queries=3, cursor="MA==", condition=lambda m: m["tags"] and "Crypto" in m["tags"]):
    found_any = False
    crypto_markets = []

    queries_made = 0
    while (queries_made < max_queries) and (len(crypto_markets) < limit):
        print(f"Fetching markets with cursor {cursor}")

        markets = poly_client.get_markets(next_cursor=cursor)
        cursor = markets["next_cursor"]

        print(f"Loaded {len(markets["data"])} markets")

        data = [m for m in markets["data"] if condition(m)]
        if data:
            if not found_any:
                print(f"First cursor with data found: {cursor}")
                found_any = True

        print(f"Found {len(data)} crypto markets")

        crypto_markets += data
        queries_made += 1

        time.sleep(0.5)

    markets = poly_client.get_markets()

    if not crypto_markets:
        print("No crypto markets found")
    else:
        print(f"Crypto markets found ({len(crypto_markets)})")
    
    print(f"Next cursor: {cursor}")

    return crypto_markets


def is_btc_related_market(m):
    return "market_slug" in m.keys() and ("btc" in m["market_slug"])

# def is_

markets = find_crypto_markets(limit=10, cursor="OTAwMA==", condition=is_btc_related_market)

Fetching markets with cursor OTAwMA==
Loaded 500 markets
First cursor with data found: OTUwMA==
Found 3 crypto markets
Fetching markets with cursor OTUwMA==
Loaded 500 markets
Found 2 crypto markets
Fetching markets with cursor MTAwMDA=
Loaded 500 markets
Found 1 crypto markets
Crypto markets found (6)
Next cursor: MTA1MDA=


In [6]:
markets[0]

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False,
 'accepting_order_timestamp': '2024-07-01T18:48:13Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0x7c1a2610911ea83b5e9a44d89201635c69919324d45e8deb92ef422b1dddc12b',
 'question_id': '0x3cc03c5ebbdcb5ba3b334e5913d49028e39b8fb6bb90c6921a82be5a15673d8b',
 'question': 'Will $BTC reach 70k in July?',
 'description': 'This market will immediately resolve to "Yes" if any Binance 1 minute candle for Bitcoin (BTCUSDT) between July 1, 2024, 00:00 and July 31, 2024, 23:59 in the ET timezone has a final "High" price of $70,000.00 or higher. Otherwise, this market will resolve to "No."\n\nThe resolution source for this market is Binance, specifically the BTCUSDT "High" prices available at https://www.binance.com/en/trade/BTC_USDT, with the chart settings on "1m" for one-minute candles selected on the top bar.\n\nPlease note that the outcome of this market depe

In [7]:
[("https://polymarket.com/event/" + m["market_slug"], m["condition_id"]) for m in markets]


[('https://polymarket.com/event/will-btc-reach-70k-in-july',
  '0x7c1a2610911ea83b5e9a44d89201635c69919324d45e8deb92ef422b1dddc12b'),
 ('https://polymarket.com/event/will-btc-reach-80k-in-july',
  '0x10bfd2053bfe798aa0fec72fc4aaf465b17b60afb453919ad2f366770042a8d7'),
 ('https://polymarket.com/event/will-btc-reach-75k-in-july',
  '0x409bbbd78d6bf45cd67048fd9ef80f473152f82aef9f3dd275a616af7474cab7'),
 ('https://polymarket.com/event/will-btc-hit-50k-or-70k-first',
  '0xf2830173b5e239e7db0509d4489105107225101cce05b41c1818b4b78e00d6e8'),
 ('https://polymarket.com/event/will-btc-or-eth-reach-all-time-high-first-2024',
  ''),
 ('https://polymarket.com/event/will-a-presidential-candidate-say-mog-at-btc-conference',
  '0x69e439c06b8ce7575d2dcb209a79a362981ec2c949cee41123e5c248807b9ed1')]

In [8]:
market = poly_client.get_market("0xa9444a9067fb2958ac36dd3f0ea25c07da5c39dc595643280b5ec037334b75de")
market

{'enable_order_book': True,
 'active': True,
 'closed': False,
 'archived': False,
 'accepting_orders': True,
 'accepting_order_timestamp': '2025-01-07T16:59:09Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.01,
 'condition_id': '0xa9444a9067fb2958ac36dd3f0ea25c07da5c39dc595643280b5ec037334b75de',
 'question_id': '0xb6a17e6442f360824cc866b24ef43b20598fc2eebe395cebd189c48f33b5d258',
 'question': 'Bitcoin up in Q1 2025?',
 'description': 'This market will resolve to "Up" if the Binance 1 minute candle for BTCUSDT on 31 March \'25 23:59 in the ET timezone has a final “close” price is higher than 93,445.45.\n\nThis market will resolve to "Down" if the Binance 1 minute candle for BTCUSDT on 31 March \'25 23:59 in the ET timezone has a final “close” price is lower than 93,445.45.\n\nThis market will resolve to "50-50" if the Binance 1 minute candle for BTCUSDT on 31 March \'25 23:59 in the ET timezone has a final “close” price is of precisely 93,445.45.\n\nThe resolution source for thi

In [9]:
markets[3]

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False,
 'accepting_order_timestamp': '2024-07-12T17:34:12Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0xf2830173b5e239e7db0509d4489105107225101cce05b41c1818b4b78e00d6e8',
 'question_id': '0x53de58d6afe1f85a7cce005214214b10c5c171ac99be33d0c5f58cd8ec4694eb',
 'question': 'Will $BTC hit $50k or $70k first?',
 'description': 'This is a market on whether Bitcoin ($BTC) will first reach $50,000.00 USD or $70,000.00 USD between July 12, 2024, 12 PM ET, and December 31, 2024, 11:59 PM ET.\n\nThis market will resolve to "50k" if $BTC drops to $50,000.00 USD or less first within the market timeframe. This market will resolve to "70k" if $BTC reaches $70,000.00 or more first within the market timeframe.\n\nIf the price of $BTC neither drops to $50,000.00 or below nor reaches $70,000.00 or more within the market timeframe, this market will resolve 50-50.\n\nThe re

In [1]:
poly_client.get_timeseries(
    "0x480691cf1edbe926d4efcabfc084e8f60558916b1de183472aa0552ef8716b50",
    720,
)

NameError: name 'poly_client' is not defined

In [22]:
import poly_api

importlib.reload(poly_api)

condition_id = "0x906a2b90f30c62e09dce56fababafafbc4338cb8dac94ac6c3038327d0447180"

ts_options = poly_api.TSOptions(
    60,
    (datetime.fromisoformat("2025-01-15"), datetime.fromisoformat("2025-01-29")),
)
df = poly_api.get_poly_timeseries(
    condition_id,
    ts_options,
)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336 entries, 2025-01-15 00:00:00 to 2025-01-28 23:00:00
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Yes     336 non-null    float64
 1   No      336 non-null    float64
dtypes: float64(2)
memory usage: 7.9 KB


In [23]:
df.head()

,Yes,No
date,,
2025-01-15 00:00:00,0.0095,0.9905
2025-01-15 01:00:00,0.0095,0.9905
2025-01-15 02:00:00,0.0095,0.9905
2025-01-15 03:00:00,0.0095,0.9905
2025-01-15 04:00:00,0.0100,0.9900


In [5]:
market = poly_client.get_market("0x7d08ca866cc7f0eff0336357de08d747c235d8594b1f58d2425f8a0c933c5baa")
market

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False,
 'accepting_order_timestamp': '2025-01-31T20:14:09Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0x7d08ca866cc7f0eff0336357de08d747c235d8594b1f58d2425f8a0c933c5baa',
 'question_id': '0xc578cb165ac523bfed6bfb60fac741f939b4e173952373c34b5d6504168544f3',
 'question': 'Will Bitcoin reach $150,000 by February 28, 2025?',
 'description': 'This market will immediately resolve to "Yes" if any Binance 1 minute candle for Bitcoin (BTCUSDT) between February 1, 2025, 00:00 and February 28, 2025, 23:59 in the ET timezone has a final "High" price of $150,000 or higher. Otherwise, this market will resolve to "No."\n\nThe resolution source for this market is Binance, specifically the BTCUSDT "High" prices available at https://www.binance.com/en/trade/BTC_USDT, with the chart settings on "1m" for one-minute candles selected on the top bar.\n\nPlease note that the 

In [17]:
from datetime import datetime, timedelta

import poly_api
importlib.reload(poly_api)

"0x480691cf1edbe926d4efcabfc084e8f60558916b1de183472aa0552ef8716b50,0x7d08ca866cc7f0eff0336357de08d747c235d8594b1f58d2425f8a0c933c5baa,0x6e5a58ad9335fb9334ea721634739e43d877af1398df97517fbc40abea4ec936,0x45060e3ad88e835074c1d7853d36d504c6f0d00120090bfe607b7dc5afdcb554,0x00661b4efa15d1ae67324e24f80fa767af020110b5db4b42ae5f528165754c31,0xbc259989970aa172c5caf59b1cb9f4c06271382a37b2b64d11864d05d15eb140,0x4165a276c3f1feb7e905c011925ff002c5bdfa62c81acebc2aae23a2642c8a4e,0xf36b89b9382415b3b6204188e5b69afbeff70c99c54b294c6dd63d00ed9ddbad,0xf79f84f044be0ea23cd3bcd33fc886a44fd3ae595f2dd59da3d0b7492897c0aa,0x3f927aff2c902bba0bd543aa984b60ed94b3a01eefaf834dfe9e964d42855148,0x87d9d0470879bb3b1ccb05e1ce06fc8f760c24a3dcb0dd677d42c4e95bbcd866,0x9d87adf2dbb64b312d9b7823f155644c6189b28c260e877396c40229e676ef4b,0x35da12adf7cbaf89e5b2caf72ffded06da30e4b889813c570969218382a73be5"
market_ids = "0x480691cf1edbe926d4efcabfc084e8f60558916b1de183472aa0552ef8716b50,0x7d08ca866cc7f0eff0336357de08d747c235d8594b1f58d2425f8a0c933c5baa,0x6e5a58ad9335fb9334ea721634739e43d877af1398df97517fbc40abea4ec936,0x45060e3ad88e835074c1d7853d36d504c6f0d00120090bfe607b7dc5afdcb554,0x00661b4efa15d1ae67324e24f80fa767af020110b5db4b42ae5f528165754c31,0xbc259989970aa172c5caf59b1cb9f4c06271382a37b2b64d11864d05d15eb140,0x4165a276c3f1feb7e905c011925ff002c5bdfa62c81acebc2aae23a2642c8a4e,0xf36b89b9382415b3b6204188e5b69afbeff70c99c54b294c6dd63d00ed9ddbad,0xf79f84f044be0ea23cd3bcd33fc886a44fd3ae595f2dd59da3d0b7492897c0aa,0x3f927aff2c902bba0bd543aa984b60ed94b3a01eefaf834dfe9e964d42855148,0x87d9d0470879bb3b1ccb05e1ce06fc8f760c24a3dcb0dd677d42c4e95bbcd866,0x9d87adf2dbb64b312d9b7823f155644c6189b28c260e877396c40229e676ef4b,0x35da12adf7cbaf89e5b2caf72ffded06da30e4b889813c570969218382a73be5".split(
    ","
)

ts_options = poly_api.TSOptions(15, (datetime.now() - timedelta(days=30), datetime.now()))

market_data = poly_api.load_markets_data(market_ids, ts_options)
market_data = market_data[market_data.index <= '2025-02-27']

market_data.info()

Didn't find any data for market `0xf36b89b9...`,  slug: `will-bitcoin-dip-to-100000-by-february-28-2025
Didn't find any data for market `0xf79f84f0...`,  slug: `will-bitcoin-dip-to-95000-by-february-28-2025
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1300 entries, 2025-02-13 11:15:00 to 2025-02-27 00:00:00
Freq: 15min
Data columns (total 11 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   will-bitcoin-reach-200000-by-february-28-2025  1300 non-null   float64
 1   will-bitcoin-reach-150000-by-february-28-2025  1300 non-null   float64
 2   will-bitcoin-reach-140000-by-february-28-2025  1300 non-null   float64
 3   will-bitcoin-reach-130000-by-february-28-2025  1300 non-null   float64
 4   will-bitcoin-reach-120000-by-february-28-2025  1300 non-null   float64
 5   will-bitcoin-reach-115000-by-february-28-2025  1300 non-null   float64
 6   will-bitcoin-reach-110000

In [18]:
market_data.head()

,will-bitcoin-reach-200000-by-february-28-2025,will-bitcoin-reach-150000-by-february-28-2025,will-bitcoin-reach-140000-by-february-28-2025,will-bitcoin-reach-130000-by-february-28-2025,will-bitcoin-reach-120000-by-february-28-2025,will-bitcoin-reach-115000-by-february-28-2025,will-bitcoin-reach-110000-by-february-28-2025,will-bitcoin-dip-to-90000-by-february-28-2025,will-bitcoin-dip-to-85000-by-february-28-2025,will-bitcoin-dip-to-75000-by-february-28-2025,will-bitcoin-dip-to-60000-by-february-28-2025
date,,,,,,,,,,,
2025-02-13 11:15:00,0.004,0.0085,0.014,0.0255,0.045,0.085,0.195,0.465,0.195,0.0345,0.0115
2025-02-13 11:30:00,0.004,0.0085,0.014,0.0255,0.040,0.085,0.175,0.465,0.195,0.0345,0.0115
2025-02-13 11:45:00,0.004,0.0085,0.014,0.0255,0.040,0.085,0.170,0.475,0.200,0.0395,0.0115
2025-02-13 12:00:00,0.004,0.0085,0.014,0.0255,0.040,0.075,0.170,0.490,0.205,0.0420,0.0115
2025-02-13 12:15:00,0.004,0.0080,0.014,0.0255,0.040,0.075,0.170,0.490,0.205,0.0420,0.0115


In [19]:
market_data.attrs

{'outcomes': {'will-bitcoin-reach-200000-by-february-28-2025': 'no',
  'will-bitcoin-reach-150000-by-february-28-2025': 'no',
  'will-bitcoin-reach-140000-by-february-28-2025': 'no',
  'will-bitcoin-reach-130000-by-february-28-2025': 'no',
  'will-bitcoin-reach-120000-by-february-28-2025': 'no',
  'will-bitcoin-reach-115000-by-february-28-2025': 'no',
  'will-bitcoin-reach-110000-by-february-28-2025': 'no',
  'will-bitcoin-dip-to-90000-by-february-28-2025': 'yes',
  'will-bitcoin-dip-to-85000-by-february-28-2025': 'yes',
  'will-bitcoin-dip-to-75000-by-february-28-2025': 'no',
  'will-bitcoin-dip-to-60000-by-february-28-2025': 'no'}}